In [ ]:
import pyodbc
import os
from IPython.core.display import display
from dotenv import load_dotenv
import pandas as pd

Connecting to SQL Server

In [ ]:
load_dotenv('credentials.env')

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER='+os.environ.get('server')+';'
    'DATABASE='+os.environ.get('database')+';'
    'UID='+ os.environ.get('uid') +';'
    'PWD='+ os.environ.get('pwd')+';')

1. Retrieve the products (and their respective storage locations) that are running low (under 50 units).

In [ ]:
low_inventory_df = pd.read_sql("SELECT l.Name AS Location, "
                               "p.Name AS Product, "
                               "SUM(prod_inv.Quantity) AS TotalQuantity "
                               "FROM Production.ProductInventory prod_inv "
                               "INNER JOIN Production.[Location] l "
                               "ON prod_inv.LocationID = l.LocationID "
                               "INNER JOIN Production.Product p "
                               "ON prod_inv.ProductID = p.ProductID "
                               "GROUP BY l.Name, p.Name "
                               "HAVING SUM(prod_inv.Quantity) < 50 "
                               "ORDER BY Location, Product;", conn)

display(low_inventory_df)

2. Retrieve all products (Name and Model) and their inventory locations where safety stock level falls below or equal to the number of available units.

In [ ]:
products_to_order_df = pd.read_sql('SELECT l.Name AS Location, '
                 'p.Name AS ProductName, '
                 'pm.Name AS Model, '
                 'p.SafetyStockLevel, '
                 'p.ReorderPoint, '
                 'TotalAvailableQty, '
                 'p.ReorderPoint - TotalAvailableQty AS UnitsToReorder '
                 'FROM Production.Product p '
                 'INNER JOIN (SELECT ProductID, '
                 'LocationID, '
                 'SUM(Quantity) OVER(PARTITION BY ProductID) AS TotalAvailableQty '
                 'FROM Production.ProductInventory) prod_inv '
                 'ON prod_inv.ProductID = p.ProductID '
                 'INNER JOIN Production.ProductModel pm '
                 'ON pm.ProductModelID = p.ProductModelID '
                 'INNER JOIN Production.[Location] l '
                 'ON prod_inv.LocationID = l.LocationID '
                 'WHERE TotalAvailableQty <= ReorderPoint '
                 'ORDER BY Location DESC;', conn)

display(products_to_order_df)

3. Retrieve all products (ProductID, Name, Model) that do not have any orders.

In [13]:
orders_check_df = pd.read_sql("SELECT p.Name AS ProductName, "
                                    "pm.Name AS Model "
                                    "FROM Production.Product p "
                                    "INNER JOIN Production.ProductModel pm "
                                    "ON pm.ProductModelID = p.ProductModelID "
                                    "INNER JOIN Purchasing.ProductVendor pv "
                                    "ON pv.ProductID = p.ProductID "
                                    "GROUP BY p.Name, pm.Name "
                                    "HAVING SUM(pv.OnOrderQty) IS NULL "
                                    "ORDER BY p.Name ASC;", conn)
display(orders_check_df)


,ProductName,Model
0,Front Brakes,Front Brakes
1,HL Mountain Tire,HL Mountain Tire
2,HL Road Pedal,HL Road Pedal
3,HL Road Tire,HL Road Tire
4,HL Touring Seat/Saddle,HL Touring Seat/Saddle
5,LL Mountain Seat/Saddle,LL Mountain Seat/Saddle 2
6,LL Mountain Tire,LL Mountain Tire
7,LL Road Seat/Saddle,LL Road Seat/Saddle 1
8,LL Road Tire,LL Road Tire
9,LL Touring Seat/Saddle,LL Touring Seat/Saddle
